In [ ]:
from __future__ import print_function

import numpy as np
import glob
import platform


## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        

from clustered_SNe.analysis.constants import m_proton, pc, yr, M_solar, \
                                   metallicity_solar
from clustered_SNe.analysis.parse import Overview, RunSummary, \
                                         Inputs, \
                                         get_full_id_from_partial_id, \
                                         checkpoint_num_from_filename

from clustered_SNe.analysis.database_helpers import session, \
                                                Simulation, \
                                                Simulation_Inputs, \
                                                Simulation_Status


In [ ]:
if ("hyades" in platform.node()) or ("eudora" in platform.node()):
    data_dir = "../cluster_parameter_study/"
else:
    data_dir = "../saved_runs/cluster_parameter_study/"

In [ ]:
energy_ids = [
    "b5f8a507-0bbe-422e-8ee6-2f6e53813027",
"0522ca33-c440-4384-ba16-2db6998fc8d0",
"7ddb48b1-ca9d-4c87-808a-a5fa6c59b687",
"3cf9d3f4-b657-4437-a59f-324de0b52763",
"2af1f468-1aaa-471f-beec-d94c328a445b",
"51e34873-903a-4a3b-8b27-26830709c9b6",
"20b4fccc-e53f-48ca-894b-3ee282bad8ea",
"f2a0cce6-88ca-4bfc-b6e9-be77356ee0bb",
"741000c8-441e-4361-bbcc-bed2ac836ad8",
"468f39c5-c4ad-472b-878e-e468224cac52",
]

resolution_ids = [
    "26e43ef9-1710-4629-96fa-6ac055930dad",
"9f8d633d-40e7-4c7b-b309-37b3218fcd1a",
"59b7f2b4-ed74-48bc-ab18-7a860043a7de",
"2af1f468-1aaa-471f-beec-d94c328a445b",
"c381b033-8dfe-4ab9-acb2-5a28fc49e9f4",
"7ddb48b1-ca9d-4c87-808a-a5fa6c59b687",
"7d54ea03-7abd-429b-9438-39b0dfcbbe8f",
"3cf9d3f4-b657-4437-a59f-324de0b52763",
"6a3b2459-9dcc-424c-a836-c23575424395",
"741000c8-441e-4361-bbcc-bed2ac836ad8",
"468f39c5-c4ad-472b-878e-e468224cac52",
"f22ff92d-fd0e-4a13-8372-7275e0bf41ed",
"372eac02-4b98-4434-86ab-f7adaf4cbb11",
]

ids = []
for rid in resolution_ids:
    if rid in energy_ids:
        continue
    overview = Overview(os.path.join(data_dir, rid + "_overview.dat"))
    if overview.SNe_times.size > 1:
        continue
    ids.append(rid)

In [ ]:
restart_dir = "restart_resolution"

qsub_systems = set(["SGE", "PBS"])
qsub_system = "PBS"
assert(qsub_system in qsub_systems)
    
f_submit = open("submit_script", mode="w")
f_submit.write("#!/bin/bash \n")

f_delete = open("delete_script", mode="w")
f_delete.write("#!/bin/bash \n")
f_delete.write("mkdir -p _tmp_ \n")


f_copy = open("copy_script", mode="w")
f_copy.write("#!/bin/bash \n")
f_copy.write("mkdir -p ../" + restart_dir + "\n")

for id in ids:

    print("id: ", id)
        
    first_checkpoint_filename  = id + "_checkpoint_0000.dat"
    
    restart_checkpoints_needed = 99
    
    restart_time_needed = 3e-2 * 3e15
    print("extra time needed: ", restart_time_needed, "[s]")

    print("")

    CFL = .025

    f_submit.write("qsub " +
                       "-v RESTART_ID=" + id + ","
                       + "RESTART_N_CHECKPOINTS=" + str(restart_checkpoints_needed) + ","
                       + "RESTART_DELTA_TIME=" + str(restart_time_needed) + ","
                       + "RESTART_CFL=" + str(CFL)
                    + " ../scripts/" + qsub_system + "/restart.batch.env_variables \n")
    
    f_delete.write("mv " + first_checkpoint_filename + " _tmp_/ \n")
    f_delete.write("rm -f " + id + "_checkpoint_*.dat \n")
    f_delete.write("mv _tmp_/" + first_checkpoint_filename + " . \n")
    
    f_copy.write("cp " + first_checkpoint_filename + " ../" + restart_dir + " \n")
    f_copy.write("cp " + id + "_overview.dat       " + " ../" + restart_dir + " \n")
    f_copy.write("cp " + id + "_SNe.dat            " + " ../" + restart_dir + " \n")
    f_copy.write("cp " + id + "_inputs.dat         " + " ../" + restart_dir + " \n")
     
f_delete.write("rm -rf _tmp_ \n")
f_submit.close()
f_copy.close()
f_delete.close()

In [ ]:
for id in ids:
#     full_id = get_full_id_from_partial_id(data_dir, id)
    session.query(Simulation_Status).filter(Simulation_Status.id == id).update({"status":"Running"})
    sim_status = session.query(Simulation_Status).get(id)
    print(sim_status.status)
    print(id)

In [ ]:
session.commit()